In [ ]:
import nltk
nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.7MB 5.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=180b62e201e8c6ec2fa32238a181263b98cb50c16d5a0a76f7be931a5a5cc6ec
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Import Dataset
df = pd.read_csv('tweets_only.csv')
#df = pd.read_csv('http://falhazmi.com/tweets_only.csv')
df.rename(columns={'tweet': 'content'},inplace=True)
#df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:10])

['A la llegada, sorpresón de las familias y lágrimas y más lágrimas. De pena y '
 'de felicidad. Hasta aquí llega el recorrido de DondeTodoSucede. Fue una '
 'aventura magnífica llena de amor y cariño mutuo. No se les olvidará nunca. '
 'pic.twitter.com/GOhrBTqyLZ',
 'Por la tarde, a Cieza a hacer rafting en el descenso a Blanca. Remando y '
 'trabajando en equipo hasta el último día. Y alguna bajadita que nos puso el '
 'estómago lleno de mariposas 😅😅 pic.twitter.com/xZTkQZMgtL',
 'Por la mañana, a Tabarca a hacer snorkel. Qué pasada. Una maravilla natural '
 'aquí al lado de casa. Alumnado que no siquiera había montado en barco (más '
 'allá del ferry que trajo a muchos a este país) y muchos que no sabían nadar '
 'y, confiando en nosotros, lo gozaron fuerte. pic.twitter.com/cMqa4WsPJ9',
 'Ayer intentamos que la clase de DondeTodoSucede tuviese una despedida digna '
 'y acorde a lo felices que nos han hecho estos años. Intentamos que fuese '
 'completamente sorpresa el día y la hora.

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:10])

[['la', 'llegada', 'sorpreson', 'de', 'las', 'familias', 'lagrimas', 'mas', 'lagrimas', 'de', 'pena', 'de', 'felicidad', 'hasta', 'aqui', 'llega', 'el', 'recorrido', 'de', 'dondetodosucede', 'fue', 'una', 'aventura', 'magnifica', 'llena', 'de', 'amor', 'carino', 'mutuo', 'no', 'se', 'les', 'olvidara', 'nunca', 'pic', 'twitter', 'com', 'gohrbtqylz'], ['por', 'la', 'tarde', 'cieza', 'hacer', 'rafting', 'en', 'el', 'descenso', 'blanca', 'remando', 'trabajando', 'en', 'equipo', 'hasta', 'el', 'ultimo', 'dia', 'alguna', 'bajadita', 'que', 'nos', 'puso', 'el', 'estomago', 'lleno', 'de', 'mariposas', 'pic', 'twitter', 'com', 'xztkqzmgtl'], ['por', 'la', 'manana', 'tabarca', 'hacer', 'snorkel', 'que', 'pasada', 'una', 'maravilla', 'natural', 'aqui', 'al', 'lado', 'de', 'casa', 'alumnado', 'que', 'no', 'siquiera', 'habia', 'montado', 'en', 'barco', 'mas', 'alla', 'del', 'ferry', 'que', 'trajo', 'muchos', 'este', 'pais', 'muchos', 'que', 'no', 'sabian', 'nadar', 'confiando', 'en', 'nosotros', 'l

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print([g for g in trigram_mod[bigram_mod[data_words[0]]] if '_'  in g])

[]


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
#!python3 -m spacy download es

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('es', disable=['parser', 'ner'])
nlp = spacy.load(r'C:/Users/lubna/anaconda3/Lib/site-packages/es_core_news_md/es_core_news_md-3.0.0')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:14])

[['llegado', 'sorpreson', 'familia', 'lagrima', 'lagrima', 'pena', 'felicidad', 'aqui', 'llegar', 'recorrido', 'dondetodosucedir', 'aventura', 'magnifico', 'lleno', 'amor', 'carino', 'mutuo', 'olvidar', 'nunca', 'pic', 'twitter', 'com', 'gohrbtqylz'], ['tarde', 'cieza', 'hacer', 'rafting', 'descenso', 'blanco', 'remar', 'trabajar', 'equipo', 'ultimo', 'dia', 'bajadita', 'poner', 'estomago', 'lleno', 'mariposa', 'pic', 'twitter', 'xztkqzmgtl'], ['manana', 'tabarca', 'hacer', 'snorkel', 'pasado', 'maravilla', 'natural', 'aqui', 'lado', 'casa', 'alumnado', 'siquiera', 'montar', 'barco', 'mas', 'allo', 'ferry', 'traer', 'pai', 'sabian', 'nadar', 'confiar', 'gozar', 'fuerte', 'pic', 'twitter', 'com', 'wspj'], ['ayer', 'intentar', 'clase', 'dondetodosucede', 'despedido', 'digno', 'acorde', 'feliz', 'hecho', 'ano', 'intentar', 'completamente', 'sorpresa', 'dia', 'hora', 'casi', 'conseguir', 'desconociar', 'venian', 'empezar', 'camiseta', 'recordar', 'pic', 'twitter', 'com', 'ppadfxkpg'], ['pr

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:14])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(12, 1), (18, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(1, 1), (4, 1), (18, 1), (21, 1), (29, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(4, 1), (18, 1), (21, 1), (26, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1)], [(62, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(4, 1), (18, 1), (21, 1), (41, 1), (46, 1), (63, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.173*"mejor" + 0.125*"nuevo" + 0.099*"llegar" + 0.092*"felicidad" + '
  '0.064*"equipo" + 0.063*"pena" + 0.043*"colegio" + 0.025*"dar él" + '
  '0.021*"olvidar" + 0.019*"llegado"'),
 (1,
  '0.231*"asi" + 0.097*"trabajar" + 0.094*"mundo" + 0.074*"libro" + '
  '0.069*"conocer" + 0.067*"feliz" + 0.048*"actividad" + 0.048*"ayuda" + '
  '0.009*"breve" + 0.008*"cantar"'),
 (2,
  '0.230*"hoy" + 0.217*"querer" + 0.135*"trabajo" + 0.063*"faltar" + '
  '0.035*"graci" + 0.025*"mantener" + 0.009*"cambiado" + 0.007*"cercano" + '
  '0.004*"respirar" + 0.000*"gracias"'),
 (3,
  '0.296*"mas" + 0.254*"hacer" + 0.081*"tiempo" + 0.079*"coronavirus" + '
  '0.057*"igual" + 0.034*"gustar" + 0.022*"yomequedoencasa" + 0.020*"padre" + '
  '0.016*"quedateencás" + 0.011*"noche"'),
 (4,
  '0.144*"aqui" + 0.117*"curso" + 0.100*"aun" + 0.047*"hilo" + 0.045*"hacer '
  'él" + 0.042*"alumnado" + 0.041*"mensaje" + 0.029*"futuro" + 0.024*"andar" + '
  '0.021*"deporte"'),
 (5,
  '0.119*"buen" + 0.109*"claro" + 0

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
#coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
#pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.487004 -0.244734       1        1  27.823007
13     0.466625  0.159282       2        1  10.122761
3     -0.477626  0.011359       3        1   7.946314
14    -0.381879 -0.252483       4        1   7.502298
6      0.188673 -0.378670       5        1   5.614852
16    -0.076474 -0.434830       6        1   5.568953
5     -0.207164 -0.277646       7        1   3.902056
12     0.172798 -0.173698       8        1   3.881886
4      0.062911  0.388261       9        1   3.335718
7      0.008804 -0.253062      10        1   3.024632
9      0.277813 -0.038327      11        1   2.953544
0     -0.128578  0.402685      12        1   2.951599
1      0.248920  0.166296      13        1   2.889925
19    -0.312733  0.222110      14        1   2.831205
2      0.170060  0.291192      15        1   2.517692
11    -0.130460  0.244669      16        1   2.129992
8     -0.232160 -0.026913      17        1   1.687609
15    -0.154537  0.084820      18        1   1.686402
17     0.050589  0.087971      19        1   1.171108
10    -0.032586  0.021718      20        1   0.458448, topic_info=           Term           Freq          Total Category  logprob  loglift
21      twitter  368236.000000  368236.000000  Default  30.0000  30.0000
127       https  163844.000000  163844.000000  Default  29.0000  29.0000
4           com  272799.000000  272799.000000  Default  28.0000  28.0000
18          pic  234930.000000  234930.000000  Default  27.0000  27.0000
573      status  125904.000000  125904.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
724   educacion       0.048263       2.134896  Topic20 -12.7557   1.5956
1009        www       0.048263       2.135229  Topic20 -12.7557   1.5954
741      verdad       0.048263       2.134443  Topic20 -12.7557   1.5958
904       ahora       0.048263       2.135314  Topic20 -12.7557   1.5954
520     gracias       0.048263       2.136022  Topic20 -12.7557   1.5950

[1010 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
420      14  0.999456            abrir
86       13  0.999570        actividad
373       3  0.998476           activo
609       2  0.999587          acuerdo
296       5  0.997851       adaptar él
...     ...       ...              ...
668      20  0.997834             vivo
223       7  0.999825           volver
419      18  0.981579           votado
483       3  0.999617  yomequedoencasa
161       8  0.999825         youtu_be

[307 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 14, 4, 15, 7, 17, 6, 13, 5, 8, 10, 1, 2, 20, 3, 12, 9, 16, 18, 11])

## HDP

In [ ]:
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

In [ ]:
hdpmodel = HdpModel(corpus=corpus, id2word=id2word, random_state= 30)


In [ ]:
def display_topics(model, model_type="lda"):
    for topic_idx, topic in enumerate(model.print_topics()):
        print ("Topic %d:" % (topic_idx))
        if model_type== "hdp":
            print (" ".join(re.findall( r'\*(.[^\*-S]+).?', topic[1])), "\n")
        else:
            print (" ".join(re.findall( r'\"(.[^"]+).?', topic[1])), "\n")

In [ ]:
# hdpmodel.show_topics() 

display_topics(hdpmodel, model_type="hdp")

Topic 0:
twitter  com  https  mas  hacer  status  pic  ver  decir  dia 

Topic 1:
twitter  com  pic  https  status  mas  hacer  ver  dia  mes 

Topic 2:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 3:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 4:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 5:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 6:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 7:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 8:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 9:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 10:
twitter  com  pic  https  status  mas  hacer  ver  decir  dia 

Topic 11:
twitter  com  pic  https  status  mas  hacer  ver  decir  gracia 

Topic 12:
twitter  com  pic  https  status  mas  hacer  ver  decir  gracia 

Topic 13:
twitter  com  pic  https  status  

## LSI

In [ ]:
lsimodel = LsiModel(corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
display_topics(lsimodel)  # Showing the topics

Topic 0:
twitter com pic https status mas hacer dia ver www 

Topic 1:
https pic status twitter com www mas youtu_be pduchement html 

Topic 2:
mas hacer ver decir dia dar com ano status solo 

Topic 3:
hacer mas ver decir ano dia bien cosa solo falta 

Topic 4:
com twitter status www https pic html youtube educacion coronavirus 

Topic 5:
https ver status hacer com pic mas www dia decir 

Topic 6:
ver https hacer pic status decir com mas www dar 

Topic 7:
ver decir dia dar hacer mas solo alumno clase tambien 

Topic 8:
dia decir hoy https pic status gracia ver querer gracias 

Topic 9:
dar decir dia gracia ver clase gracias alumno bien tambien 

Topic 10:
gracia dar gracias compartir clase bien decir abrazo esperar siempre 

Topic 11:
dar gracia gracias solo decir tambien alumno dia educacion asi 

Topic 12:
gracias dar gracia ano bien solo decir alumno creer ahora 

Topic 13:
ano querer tambien asi solo creer llevar ahora hoy educacion 

Topic 14:
bien asi educacion alumno solo quer

## LDA

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=30, id2word=id2word)

In [ ]:
display_topics(ldamodel) 

Topic 0:
querer pasar venir momento seguro escuchar palabra mas juego dicho 

Topic 1:
ir llevar padre hijo tocar ahi preguntar crear lado escribir 

Topic 2:
serio mano punto llamar madre necesario cambiar pequeno dificil malo 

Topic 3:
compartir gran trabajo volver hora interesante espana listo nota nombre 

Topic 4:
gracias bien poner parecer semana aun forma companero casi dentro 

Topic 5:
video quedateencás app lunes contagio parar responsabilidad tweet internet 

Topic 6:
tanto quedar profesor usar mas colegio encontrar despues mayor facil 

Topic 7:
ver mas vez solo menos bueno gustar hacer conocer visto 

Topic 8:
dar igual cole estar vuelta favor saludo noche gusto luz 

Topic 9:
educacion hablar html dejar proyecto junto blog ayudar prof blogspot 

Topic 10:
primero trabajar recurso felicidad tomar material lugar comprar funcionar seguridad 

Topic 11:
gracia aqui idea yomequedoencasa verano duda ver él super virtual tuit 

Topic 12:
esperar acabar unico cuarentena pregunta